In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc

ModuleNotFoundError: No module named 'pymc'

In [ ]:
co2_gr = pd.read_csv('co2_gr_gl.txt',delim_whitespace=True, skiprows=62, names=['year', 'rate', 'err'])

co2_now = pd.read_csv('co2_annmean_gl.txt', delim_whitespace=True, skiprows=57, names=['year', 'co2', 'err'])

co2_200 = pd.read_csv('siple2.013.dat')

co2_1000 = pd.read_csv('lawdome.smoothed.yr75.dat', delim_whitespace=True, skiprows=22, names=['year', 'co2'])

In [2]:
co2_now.tail()

NameError: name 'co2_now' is not defined

In [3]:
co2_now = co2_now[:-3]

NameError: name 'co2_now' is not defined

In [4]:
print( co2_now['year'].dtype, co2_1000['co2'].dtype,
co2_now['co2'].dtype, co2_gr['rate'].dtype)

NameError: name 'co2_now' is not defined

In [ ]:
co2_now['year'] = pd.to_numeric(co2_now['year'])
co2_now['co2'] = pd.to_numeric(co2_now['co2'])
co2_now['co2'].dtype,co2_now['year'].dtype

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10,4))
ax2 = axs[0]
ax2.errorbar(co2_now['year'], co2_now['co2'],
#yerr=co2_now['err'],
color='SteelBlue', ls='None', elinewidth=1.5, capthick=1.5, marker='.', ms=6)
ax2.plot(co2_1000['year'], co2_1000['co2'], color='Green', ls='None', marker='.', ms=6)
ax2.plot(co2_200['year'], co2_200['co2'], color='IndianRed', ls='None', marker='.', ms=6)
ax2.axvline(1800, lw=2, color='Gray', dashes=(6,5))
ax2.axvline(co2_gr['year'][0], lw=2, color='SteelBlue', dashes=(6,5))
print(co2_gr['year'][0])
ax2.legend(['Recent', 'LAW ice core', 'SIPLE ice core'],fontsize=15, loc=2)
labels = ax2.get_xticklabels()
plt.setp(labels, rotation=33, ha='right')
ax2.set_ylabel('CO$_2$ (ppm)')
ax2.set_xlabel('Year')
ax2.set_title('Past CO$_2$')
ax1 = axs[1]
ax1.errorbar(co2_gr['year'], co2_gr['rate'], yerr=co2_gr['err'], color='SteelBlue', ls='None',
elinewidth=1.5,capthick=1.5, marker='.', ms=8)
labels = ax1.get_xticklabels()
plt.setp(labels, rotation=33, ha='right')
ax1.set_ylabel('CO$_2$ growth (ppm/yr)')
ax1.set_xlabel('Year')
ax1.set_xlim((1957,2016))
ax1.set_title('Growth rate since 1960');

In [ ]:
_ = plt.hist(co2_gr['err'], bins=20)
plt.xlabel('Uncertainty')
plt.ylabel('Count')

In [ ]:
x = co2_gr['year'].values
y = co2_gr['rate'].values
y_error = co2_gr['err'].values

In [ ]:
def model(x, y):
    slope = pymc.Normal('slope', 0.1, 1.)
    intercept = pymc.Normal('intercept', -50., 10.)
    @pymc.deterministic(plot=False)
    def linear(x=x, slope=slope, intercept=intercept):
        return x * slope + intercept
    f = pymc.Normal('f', mu=linear,
                    tau=1.0/y_error, value=y, observed=True)
    return locals()

In [ ]:
MDL = pymc.MCMC(model(x,y))
MDL.sample(5e5, 5e4, 100)

In [ ]:
y_min = MDL.stats()['linear']['quantiles'][2.5]
y_max = MDL.stats()['linear']['quantiles'][97.5]
y_fit = MDL.stats()['linear']['mean']
slope = MDL.stats()['slope']['mean']
slope_err = MDL.stats()['slope']['standard deviation']
intercept = MDL.stats()['intercept']['mean']
intercept_err = MDL.stats()['intercept']['standard deviation']

In [ ]:
mcplt.plot(MDL)

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
ols_results = smf.ols("rate ~ year", co2_gr).fit()

In [ ]:
prstd, iv_l, iv_u = wls_prediction_std(ols_results)
ols_params = np.flipud(ols_results.params)
ols_err = np.flipud(np.diag(ols_results.cov_params())**.5)

In [ ]:
print('OLS: slope:{0:.3f}, intercept:{1:.2f}'.format(*ols_params))
print('Bay: slope:{0:.3f}, intercept:{1:.2f}'.format(slope,intercept))

In [ ]:
ols_results.conf_int(alpha=0.05)

In [ ]:
MDL.stats(['intercept','slope'])

In [ ]:
plt.figure(figsize=(10,6))
plt.title('Growth rate since 1960');
plt.errorbar(x,y,yerr=y_error, color='SteelBlue', ls='None', elinewidth=1.5, capthick=1.5,
marker='.', ms=8, label='Observed')
plt.xlabel('Year')
plt.ylabel('CO$_2$ growth rate (ppm/yr)')
plt.plot(x, y_fit, 'k', lw=2, label='pymc')
plt.fill_between(x, y_min, y_max, color='0.5', alpha=0.5, label='Uncertainty')
plt.plot([x.min(), x.max()], [ols_results.fittedvalues.min(), ols_results.fittedvalues.max()], 'r', dashes=(13,2), lw=1.5, label='OLS', zorder=32)
plt.legend(loc=2, numpoints=1)